In [1]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
import snappy_utils
import os
from datetime import date
import settings
import zipfile

In [2]:
start_date = date(2019, 12, 21)
end_date = date(2019, 12, 27)
DATA_PATH = "..\\data"

### Define class for downloading Sentinel products

In [ ]:
class Sentinelsat_products:
    def __init__(self, date_start, date_finish, footprint=settings.footprint, platformname="Sentinel-3"):
        self.date_start = date_start
        self.date_finish = date_finish
        self.api = SentinelAPI(settings.sentinel_api_user, settings.sentinel_api_key, 'https://scihub.copernicus.eu/dhus')
        self.wkt_footprint = footprint
        self.products = self.query_products(self.date_start, self.date_finish)

    def query_products(self, date_start, date_finish, platformname="Sentinel-3"):
        # connect to the API
        api = SentinelAPI(settings.sentinel_api_user, settings.sentinel_api_key, 'https://scihub.copernicus.eu/dhus')

        # search by polygon, time, and Hub query keywords
        products = api.query(self.wkt_footprint, area_relation='Contains',
                            date = (self.date_start, self.date_finish),
                            platformname = 'Sentinel-3')
        return products
    
    def filter_products(self, instrument, level, p_type, timeliness):
        removed_products = []
        for product_key in self.products:
            odata = self.api.get_product_odata(product_key, full=True)
            product_instrument = odata["Instrument"]
            product_level = odata["Product level"]
            product_type = odata["Product type"]
            #mission_type = odata["Mission type"]
            product_timeliness = odata["Timeliness Category"]
            #filter only from Level 1 OLCI instrument with NTC full resolution
            conditions = (
                (product_instrument == instrument) and (p_type in product_type) 
                and product_timeliness == timeliness and product_level == level
            )
            if conditions:
                pass
                #print(instrument, product_level, product_type)
            else:
                removed_products.append(product_key)
        for key in removed_products:
            del self.products[key]

    def download_products(self, unzip=True):
        print("----------")
        downloaded_paths = []
        for key in self.products:
            file_name = self.products[key]["filename"]
            file_date = self.products[key]["summary"][:16].split("Date: ")[1]
            download_path = os.path.join(settings.data_path, file_date)
            if not os.path.exists(download_path):
                os.makedirs(download_path)
            # if it was downloaded before it won't download again            
            download_info = self.api.download(key, directory_path=download_path)
            #print(download_info)
            if unzip:
                downloaded_paths.append(os.path.join(download_path, file_name))
                zip_path = download_info["path"]
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    zip_ref.extractall(download_path)
            else:
                downloaded_paths.append(download_info["path"])
            return downloaded_paths

### Download products

In [ ]:
datos = Sentinelsat_products(start_date, end_date)
datos.filter_products(instrument="OLCI", level="L1", p_type= "FR", timeliness="Non Time Critical")
dowloaded_paths = datos.download_products()

### Make subsets of downloaded products

In [ ]:
for path in downloaded_paths:
    product = snappy_utils.read_product(path)
    subset = snappy_utils.make_subset(product, settings.footprint)
    snappy_utils.write_product(subset, os.path.join(os.path.dirname(path), "laguna.dim"), "BEAM-DIMAP")

### Reprocess downloaded data to have BEAM-DIMAP with pixel GeoCoding

In [ ]:
# iterate over the directorys containing every day's data
for directory in os.listdir(DATA_PATH):
    dir = os.path.join(DATA_PATH, directory)
    print(dir)
    index = 0
    # iterate over the content of those directorys
    for d in os.listdir(dir):
        # find the S3 downloaded data and make subset
        directory_content = os.path.join(DATA_PATH, directory, d)
        if os.path.isdir(directory_content) and ("S3" in d):
            index += 1
            dim_name = os.path.join(DATA_PATH, directory, "laguna_PixelGeoCoding") + str(index) + ".dim"
            print(dim_name)
            print(os.path.isfile(dim_name))
            #If BEAM-DIMAP is already generated, go to next day directory
            if (os.path.isfile(dim_name)):
                continue
            product = snappy_utils.read_product(directory_content)
            subset = snappy_utils.make_subset(product, settings.footprint)
            snappy_utils.write_product(subset, os.path.join(DATA_PATH, directory, "laguna_PixelGeoCoding") + str(index) + ".dim",
                                       "BEAM-DIMAP")

..\data\2019-12-21
..\data\2019-12-21\laguna_PixelGeoCoding1.dim
True
..\data\2019-12-22
..\data\2019-12-22\laguna_PixelGeoCoding1.dim
True
..\data\2019-12-22\laguna_PixelGeoCoding2.dim
True
..\data\2019-12-23
..\data\2019-12-23\laguna_PixelGeoCoding1.dim
True
..\data\2019-12-24
..\data\2019-12-24\laguna_PixelGeoCoding1.dim
True
..\data\2019-12-25
..\data\2019-12-25\laguna_PixelGeoCoding1.dim
True
..\data\2019-12-26
..\data\2019-12-26\laguna_PixelGeoCoding1.dim
True
..\data\2019-12-28
..\data\2019-12-28\laguna_PixelGeoCoding1.dim
True
..\data\2019-12-29
..\data\2019-12-29\laguna_PixelGeoCoding1.dim
True
..\data\2019-12-30
..\data\2019-12-30\laguna_PixelGeoCoding1.dim
True
..\data\2019-12-31
..\data\2019-12-31\laguna_PixelGeoCoding1.dim
True
..\data\2020-01-01
..\data\2020-01-01\laguna_PixelGeoCoding1.dim
True
..\data\2020-01-02
..\data\2020-01-02\laguna_PixelGeoCoding1.dim
True
..\data\2020-01-02\laguna_PixelGeoCoding2.dim
True
..\data\2020-01-03
..\data\2020-01-03\laguna_PixelGeoCodin

..\data\2020-03-23\laguna_PixelGeoCoding1.dim
False
Making subset
Writing BEAM-DIMAP to ..\data\2020-03-23\laguna_PixelGeoCoding1.dim
..\data\2020-03-23\laguna_PixelGeoCoding2.dim
False
Making subset
Writing BEAM-DIMAP to ..\data\2020-03-23\laguna_PixelGeoCoding2.dim
..\data\2020-03-24
..\data\2020-03-24\laguna_PixelGeoCoding1.dim
False


In [ ]:
# iterate over the directorys containing every day's data
for directory in os.listdir(DATA_PATH):
    dir = os.path.join(DATA_PATH, directory)
    print(dir)
    index = 0
    # iterate over the content of those directorys
    for d in os.listdir(dir):
        # find the S3 downloaded data and make subset
        directory_content = os.path.join(DATA_PATH, directory, d)
        if os.path.isdir(directory_content) and ("S3" in d):
            print(os.path.join(DATA_PATH, dir, d)

In [ ]:
directory_content = 
product = snappy_utils.read_product(directory_content)
subset = snappy_utils.make_subset(product, settings.footprint)
snappy_utils.write_product(subset, os.path.join(DATA_PATH, directory, "laguna_PixelGeoCoding") + str(index) + ".dim",
                                       "BEAM-DIMAP")